In [66]:
import pandas as pd
import pickle
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras import layers
from keras.optimizers import Adam
from keras.utils import to_categorical

kd = pd.read_csv('online_gaming_missing.csv')
kd.columns
kd.isnull().any()

PlayerID                     True
Age                          True
Gender                       True
Location                     True
GameGenre                    True
PlayTimeHours                True
InGamePurchases              True
GameDifficulty               True
SessionsPerWeek              True
AvgSessionDurationMinutes    True
PlayerLevel                  True
AchievementsUnlocked         True
EngagementLevel              True
dtype: bool

In [67]:
kd = kd.dropna()
kd

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
26,9026.0,21.0,Male,USA,Strategy,4.898342,0.0,Medium,12.0,118.0,71.0,26.0,High
33,9033.0,23.0,Male,USA,Simulation,18.964324,0.0,Hard,19.0,48.0,62.0,42.0,Medium
36,9036.0,34.0,Male,Europe,Action,13.776451,0.0,Hard,19.0,28.0,79.0,41.0,Medium
69,9069.0,23.0,Male,Other,Simulation,13.724648,1.0,Medium,17.0,167.0,72.0,24.0,High
72,9072.0,22.0,Male,Europe,RPG,12.082182,1.0,Easy,7.0,79.0,46.0,32.0,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40004,49004.0,35.0,Female,USA,Strategy,6.876427,1.0,Easy,1.0,139.0,4.0,6.0,Low
40005,49005.0,38.0,Male,Europe,Sports,0.360774,0.0,Easy,7.0,39.0,56.0,36.0,Medium
40007,49007.0,30.0,Male,Asia,Action,21.232485,0.0,Medium,5.0,56.0,76.0,31.0,Medium
40009,49009.0,39.0,Male,USA,Simulation,19.166112,0.0,Hard,12.0,65.0,74.0,12.0,Medium


In [68]:
def map_data(x):
    convert = x.unique()
    return x.map(dict(zip(convert, range(1,len(convert) + 1))))
for column in kd.columns:
    if kd[column].dtype == 'object':
        kd[column] = map_data(kd[column])
kd = kd.astype(int)
kd

,PlayerID,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked,EngagementLevel
26,9026,21,1,1,1,4,0,1,12,118,71,26,1
33,9033,23,1,1,2,18,0,2,19,48,62,42,2
36,9036,34,1,2,3,13,0,2,19,28,79,41,2
69,9069,23,1,3,2,13,1,1,17,167,72,24,1
72,9072,22,1,2,4,12,1,3,7,79,46,32,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40004,49004,35,2,1,1,6,1,3,1,139,4,6,3
40005,49005,38,1,2,5,0,0,3,7,39,56,36,2
40007,49007,30,1,4,3,21,0,1,5,56,76,31,2
40009,49009,39,1,1,2,19,0,2,12,65,74,12,2


In [69]:
X = kd.drop(columns=['EngagementLevel', 'PlayerID'])
y = kd['EngagementLevel']

scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled

,Age,Gender,Location,GameGenre,PlayTimeHours,InGamePurchases,GameDifficulty,SessionsPerWeek,AvgSessionDurationMinutes,PlayerLevel,AchievementsUnlocked
0,0.176471,0.0,0.000000,0.00,0.173913,0.0,0.0,0.631579,0.639053,0.714286,0.530612
1,0.235294,0.0,0.000000,0.25,0.782609,0.0,0.5,1.000000,0.224852,0.622449,0.857143
2,0.558824,0.0,0.333333,0.50,0.565217,0.0,0.5,1.000000,0.106509,0.795918,0.836735
3,0.235294,0.0,0.666667,0.25,0.565217,1.0,0.0,0.894737,0.928994,0.724490,0.489796
4,0.205882,0.0,0.333333,0.75,0.521739,1.0,1.0,0.368421,0.408284,0.459184,0.653061
...,...,...,...,...,...,...,...,...,...,...,...
4809,0.588235,1.0,0.000000,0.00,0.260870,1.0,1.0,0.052632,0.763314,0.030612,0.122449
4810,0.676471,0.0,0.333333,1.00,0.000000,0.0,1.0,0.368421,0.171598,0.561224,0.734694
4811,0.441176,0.0,1.000000,0.50,0.913043,0.0,0.0,0.263158,0.272189,0.765306,0.632653
4812,0.705882,0.0,0.000000,0.25,0.826087,0.0,0.5,0.631579,0.325444,0.744898,0.244898


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(32, activation="relu"),
    layers.Dense(3, activation="softmax")
])

# Use a lower learning rate for better convergence
optimizer = Adam(learning_rate=0.001)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
from math import ceil
print(f"Batch : {ceil(((len(X_train) - int((len(X_train) * 0.15)))) / 32)}")
print(f"Test accuracy: {test_accuracy:.2f}")

IndexError: index 3 is out of bounds for axis 1 with size 3